In [4]:
import urllib.request as urllib2
import time
from datetime import datetime
import json
from urllib.parse import urlencode, quote_plus     
import pandas as pd

In [5]:
TOP_API_URL = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'\
              'top/{lang}.{project}/all-access/{year}/{month}/{day}'

def get_traffic(year, month, day):
    '''\
    Get the traffic report for the top 1000 articles for a given day.
    TODO: Get from local file, if available
    Taken from: https://towardsdatascience.com/scraping-wikipedia-page-views-to-make-a-2020-rewind-c9bcac97fa38
    '''
    url = TOP_API_URL.format(lang='he',
                             project='wikipedia',
                             year=year,
                             month=month,
                             day=day)
    resp = urllib2.urlopen(url)
    resp_bytes = resp.read()
    data = json.loads(resp_bytes)
    articles = data['items'][0]['articles']
    return articles


In [7]:
dates_range = pd.date_range('2023-01-01','2023-04-01',freq='d')

In [8]:
lst = []
for i, date in enumerate(dates_range):
    print(date)
    df = get_traffic(dates_range[i].year, f"{dates_range[i]:%m}", f"{dates_range[i]:%d}")
    df = pd.DataFrame(df)
    df['date'] = date
    lst.append(df)
    time.sleep(1)

2023-01-01 00:00:00
2023-01-02 00:00:00
2023-01-03 00:00:00
2023-01-04 00:00:00
2023-01-05 00:00:00
2023-01-06 00:00:00
2023-01-07 00:00:00
2023-01-08 00:00:00
2023-01-09 00:00:00
2023-01-10 00:00:00
2023-01-11 00:00:00
2023-01-12 00:00:00
2023-01-13 00:00:00
2023-01-14 00:00:00
2023-01-15 00:00:00
2023-01-16 00:00:00
2023-01-17 00:00:00
2023-01-18 00:00:00
2023-01-19 00:00:00
2023-01-20 00:00:00
2023-01-21 00:00:00
2023-01-22 00:00:00
2023-01-23 00:00:00
2023-01-24 00:00:00
2023-01-25 00:00:00
2023-01-26 00:00:00
2023-01-27 00:00:00
2023-01-28 00:00:00
2023-01-29 00:00:00
2023-01-30 00:00:00
2023-01-31 00:00:00
2023-02-01 00:00:00
2023-02-02 00:00:00
2023-02-03 00:00:00
2023-02-04 00:00:00
2023-02-05 00:00:00
2023-02-06 00:00:00
2023-02-07 00:00:00
2023-02-08 00:00:00
2023-02-09 00:00:00
2023-02-10 00:00:00
2023-02-11 00:00:00
2023-02-12 00:00:00
2023-02-13 00:00:00
2023-02-14 00:00:00
2023-02-15 00:00:00
2023-02-16 00:00:00
2023-02-17 00:00:00
2023-02-18 00:00:00
2023-02-19 00:00:00


In [9]:
dat = pd.concat(lst)
dat.to_csv('daily_data.tsv')

In [58]:
top_15 = (dat
 .groupby('article')
 .agg(views = ('views', 'sum'))
 .sort_values(by = 'views', ascending = False)
 .head(25)
 .reset_index()
 .loc[lambda df: ~(df.article.str.contains('(:|עמוד_ראשי)')), :]
 .head(15)
 .replace({"הזמר_במסכה_עונה_3_(תוכנית_טלוויזיה_ישראלית)" :'הזמר_במסכה_עונה_3' })
)

C:\Users\amitl\AppData\Local\Temp\ipykernel_22660\1218370233.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  .loc[lambda df: ~(df.article.str.contains('(:|עמוד_ראשי)')), :]


In [75]:
with open('top_15.json', 'w', encoding='utf-8') as file:
    top_15.to_json(file, force_ascii=False, orient = 'index')